In [2]:
import geopandas as gpd
import os
import warnings
import ast

warnings.filterwarnings("ignore")

# Base directory path
base_dir = r"C:\Users\natda\Desktop\NatDave\Academics\PhD_NU\RESEARCH\Traffic_Stress\Boston"

# Construct file paths
roads_path = os.path.join(base_dir, "street_network.shp")
crossings_path = os.path.join(base_dir, "crossings.shp")
refuge_islands_path = os.path.join(base_dir, "osm_refuge_islands.gdb")
unassociated_refuge_islands_path = os.path.join(base_dir, "unassociated_refuge_islands.shp")

# Load shapefiles
roads = gpd.read_file(roads_path)
crossings = gpd.read_file(crossings_path)
refuge_islands = gpd.read_file(refuge_islands_path)

# Convert CRS_LEG from string to list
crossings['CRS_LEG'] = crossings['CRS_LEG'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Reproject refuge islands to match roads CRS
refuge_islands = refuge_islands.to_crs(roads.crs)

# Add 'ID' column to refuge islands and assign sequential values
refuge_islands['ID'] = range(1, len(refuge_islands) + 1)

# Buffer refuge islands by 5 meters
refuge_islands['geometry'] = refuge_islands.geometry.buffer(5)

# Find road segments that intersect with refuge islands
refuge_road_segments = roads[roads.geometry.intersects(refuge_islands.unary_union)]
refuge_road_ids = set(refuge_road_segments['unique_id'])

# Create CRS_ISLAND column and assign 1 where a refuge island exists, 0 otherwise
crossings['CRS_ISLAND'] = crossings['CRS_LEG'].apply(lambda x: 1 if any(seg in refuge_road_ids for seg in x) else 0)

# Identify unassociated refuge islands (those not linked to any road segment)
unassociated_refuge_islands = refuge_islands[~refuge_islands.intersects(refuge_road_segments.unary_union)]

# Save unassociated refuge islands to a new shapefile
unassociated_refuge_islands.to_file(unassociated_refuge_islands_path, driver='ESRI Shapefile')

# Save the updated crossings shapefile
crossings.to_file(crossings_path, driver='ESRI Shapefile')

print("Updated crossings.shp with CRS_ISLAND attribute for refuge islands.")
print(f"There are {len(unassociated_refuge_islands)} unassociated refuge islands.")

Updated crossings.shp with CRS_ISLAND attribute for refuge islands.
There are 70 unassociated refuge islands.


In [3]:
crossings['CRS_ISLAND'].value_counts()

CRS_ISLAND
0    40355
1      448
Name: count, dtype: int64

In [ ]:
unc_island = crossings[(crossings['CRS_ISLAND'] == 1) & 
          (crossings['CONTROL'].isna() | (crossings['CONTROL'] == ""))]

unc_island_path = os.path.join(base_dir, "unc_island.shp")
unc_island.to_file(unc_island_path, driver='ESRI Shapefile')

print("Filtered crossings saved to 'unc_island.shp'")

Filtered crossings saved to 'unc_island.shp'
